In [1]:
import glob
import json
import os
from typing import List

import cv2
import detectron2.data.transforms as T
import matplotlib.pyplot as plt
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    DatasetCatalog,
    DatasetMapper,
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.visualizer import Visualizer
from utils import (
    annotate_polygons,
    basic_augmentation,
    create_dataset_dicts,
    create_train_augmentations,
    move_train_test,
)
from detectron2.data.datasets import register_coco_instances

In [2]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

0

In [3]:
move_train_test("../data/bh")

In [4]:
for dataset_type in ["train", "test"]:
    DatasetCatalog.register(
        f"yolo_{dataset_type}",
        lambda d=dataset_type: create_dataset_dicts(f"../data/bh/{d}"),
    )
MetadataCatalog.get(f"yolo_{dataset_type}").thing_classes = ["yolo", "volume"]
hold_metadata_train = MetadataCatalog.get("yolo_train")
hold_metadata_test = MetadataCatalog.get("yolo_test")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
)

cfg.MODEL.DEVICE='cpu'
cfg.DATASETS.TRAIN = ("yolo_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00125
cfg.SOLVER.MAX_ITER = 50
cfg.SOLVER.STEPS = [] 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
experiment_name = "Yolo_Detection_Basic"
output_dir = "output"

cfg.OUTPUT_DIR = os.path.join(output_dir, experiment_name)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
with open(os.path.join(cfg.OUTPUT_DIR, "yolo_experiment_config.yml"), "w") as f:
    f.write(cfg.dump())
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "yolo_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # confidence threshold for testing
cfg.DETECTIONS_PER_IMAGE = 300
cfg.DATASETS.TEST = ("yolo_test",)
predictor = DefaultPredictor(cfg)

[03/30 12:43:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output\Yolo_Detection_Basic\yolo_final.pth ...


In [ ]:
for dataset_type in ["bh-phone", "sm"]:
    DatasetCatalog.register(
        f"yolo_{dataset_type}",
        lambda d=dataset_type: create_dataset_dicts(f"../data/{d}"),
    )
    MetadataCatalog.get(f"yolo_{dataset_type}").thing_classes = ["yolo", "volume"]

In [ ]:
for i, file_name in enumerate(["../data/sm/108.jpg", "../data/bh/test/_DSC9396.jpg"]):
    im = cv2.imread(file_name)
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1],
        metadata=hold_metadata_test,
        scale=0.5,
    )
    out_predictions = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    fname = file_name.split("/")[-1].split(".")[0]
    fig, ax = plt.subplots()
    ax.axis("off")
    ax.imshow(out_predictions.get_image())
    fig.savefig(f"{fname}-labeled.jpg", bbox_inches="tight", dpi=300)

In [ ]:
evaluator = COCOEvaluator(
    "yolo_test", output_dir="./output", max_dets_per_image=cfg.DETECTIONS_PER_IMAGE
)
test_loader = build_detection_test_loader(cfg, "yolo_test")
print(inference_on_dataset(predictor.model, test_loader, evaluator))

In [ ]:
for fpath in glob.glob("output/Yolo*/"):
    cfg = get_cfg()
    cfg.merge_from_file(os.path.join(fpath, "experiment_config.yml"))
    cfg.MODEL.WEIGHTS = os.path.join(fpath, "yolo_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # confidence threshold for testing
    cfg.DETECTIONS_PER_IMAGE = 300
    cfg.DATASETS.TEST = ("yolo_test", "yolo_sm", "yolo_bh-phone")
    predictor = DefaultPredictor(cfg)

    results = {}
    for dataset in cfg.DATASETS.TEST:
        evaluator = COCOEvaluator(
            dataset, output_dir="./output", max_dets_per_image=cfg.DETECTIONS_PER_IMAGE
        )
        test_loader = build_detection_test_loader(cfg, dataset)
        result = inference_on_dataset(predictor.model, test_loader, evaluator)
        results[dataset] = result

    # Dump results in model dir
    with open(os.path.join(fpath, "yolo_test_results.json"), "w") as f:
        json.dump(results, f)
    del predictor.model
    del predictor